In [1]:
import os
os.chdir('/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/src/controller/ROI')

from glob import glob
from ROI import find_ROIs_soundfile,compute_ROI_all_features
import pandas as pd
from pathlib import Path
import warnings
import librosa
import numpy as np
import maad
import soundfile as sf
import warnings
warnings.filterwarnings(action='ignore')


In [2]:
datapath = '/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/xenocanto'
annot_path = '/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/tables/annot.csv'

filelist = glob(datapath+'/**/*.wav', recursive = True)
df_data=pd.DataFrame()
for file in filelist:
    species = Path(file).parts[-2].rsplit(sep='_')[1]
    gen = Path(file).parts[-2].rsplit(sep='_')[0]
    df_data = df_data.append({'fullfilename': file,
                            'sound_id': Path(file).parts[-1][:-4],
                            'species': species,
                            'gen' : gen},
                            ignore_index=True)

df_annot = pd.read_csv(annot_path)

In [3]:
df_annot

,Unnamed: 0,record,n_roi,min_t,max_t,min_f,max_f,dt,df,APD,sound_id,gen,species
0,0,432469,1,3.669150,3.717428,2347.826,2695.652,0.0483,347.826,-64.27,432469,Acrocephalus,baeticatus
1,1,432469,2,3.753637,3.813985,2391.304,2652.174,0.0603,260.870,-61.25,432469,Acrocephalus,baeticatus
2,2,432469,3,3.886402,3.934681,2304.348,2608.696,0.0483,304.348,-58.94,432469,Acrocephalus,baeticatus
3,3,432469,4,4.013133,4.055376,2347.826,2695.652,0.0422,347.826,-61.04,432469,Acrocephalus,baeticatus
4,4,432469,5,4.115724,4.176072,2304.348,2739.130,0.0603,434.783,-60.95,432469,Acrocephalus,baeticatus
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,989,400928,59,23.113200,23.226900,11646.465,12311.977,0.1137,665.512,-58.25,400928,Cossypha,humeralis
990,990,400928,60,23.700700,23.823900,1941.077,3715.777,0.1232,1774.699,-52.38,400928,Cossypha,humeralis
991,991,400928,61,23.909200,24.013400,1941.077,3715.777,0.1042,1774.699,-51.82,400928,Cossypha,humeralis
992,992,400928,62,24.108200,24.212400,1885.618,3549.399,0.1042,1663.781,-52.37,400928,Cossypha,humeralis


In [4]:
df_annot['sound_id']=df_annot['record'].astype(str)
df_annot = df_annot[df_annot.sound_id.isin(df_data.sound_id)]

In [5]:
file = filelist[13]

In [6]:
temp, sr = librosa.load(file, sr=None)
sf.write('tmp.wav', temp, sr)
dB_max = 96
s, fs = maad.sound.load('tmp.wav')
maxAmp = np.abs(s).max()  # used to normalize
Sxx_power, tn, fn, ext = maad.sound.spectrogram(
    s/maxAmp, fs, flims=(0, 20000), display=False)
Sxx_db = maad.util.power2dB(Sxx_power) + dB_max


features = ['min_t','max_t','min_f','max_f','dt','df']

In [7]:
len(df_annot[df_annot['sound_id']==Path(file).parts[-1][:-4]])

52

In [9]:
df_roi_annot = df_annot[df_annot['sound_id']==Path(file).parts[-1][:-4]]
df_roi_annot = maad.util.format_features(df_roi_annot,tn,fn)

In [10]:
df_rois_shape = maad.features.all_shape_features(
s, fs, df_roi_annot, resolution='med', display=False)
df_rois_shape = df_rois_shape.reset_index(drop=True)

number of rois : 52


In [12]:
df_rois_shape.columns

Index(['Unnamed: 0', 'record', 'n_roi', 'min_t', 'max_t', 'min_f', 'max_f',
       'dt', 'df', 'APD', 'sound_id', 'gen', 'species', 'min_y', 'min_x',
       'max_y', 'max_x', 'shp_001', 'shp_002', 'shp_003', 'shp_004', 'shp_005',
       'shp_006', 'shp_007', 'shp_008', 'shp_009', 'shp_010', 'shp_011',
       'shp_012', 'shp_013', 'shp_014', 'shp_015', 'shp_016', 'shp_017',
       'shp_018', 'shp_019', 'shp_020', 'shp_021', 'shp_022', 'shp_023',
       'shp_024', 'shp_025', 'shp_026', 'shp_027', 'shp_028', 'shp_029',
       'shp_030', 'shp_031', 'shp_032', 'shp_033', 'shp_034', 'shp_035',
       'shp_036', 'shp_037', 'shp_038', 'shp_039', 'shp_040', 'shp_041',
       'shp_042', 'shp_043', 'shp_044', 'shp_045', 'shp_046', 'shp_047',
       'shp_048', 'centroid_y', 'centroid_x', 'duration_x', 'bandwidth_y',
       'area_xy', 'centroid_f', 'centroid_t', 'duration_t', 'bandwidth_f',
       'area_tf'],
      dtype='object')

In [186]:
# COMPUTE MAAD SPECTRAL AND TEMPORAL ALPHA FEATURES ON EACH ROI
SPECTRAL_FEATURES = ['MEANf', 'VARf', 'SKEWf', 'KURTf', 'NBPEAKS', 'LEQf',
                        'ENRf', 'BGNf', 'SNRf', 'Hf', 'EAS', 'ECU', 'ECV', 'EPS', 'EPS_KURT', 'EPS_SKEW', 'ACI',
                        'NDSI', 'rBA', 'AnthroEnergy', 'BioEnergy', 'BI', 'ROU', 'ADI', 'AEI', 'LFC', 'MFC', 'HFC',
                        'ACTspFract', 'ACTspCount', 'ACTspMean', 'EVNspFract', 'EVNspMean', 'EVNspCount',
                        'TFSD', 'H_Havrda', 'H_Renyi', 'H_pairedShannon', 'H_gamma', 'H_GiniSimpson', 'RAOQ',
                        'AGI', 'ROItotal', 'ROIcover']

TEMPORAL_FEATURES = ['ZCR', 'MEANt', 'VARt', 'SKEWt', 'KURTt',
                        'LEQt', 'BGNt', 'SNRt', 'MED', 'Ht', 'ACTtFraction', 'ACTtCount',
                        'ACTtMean', 'EVNtFraction', 'EVNtMean', 'EVNtCount']

df_annot_final = pd.DataFrame()
i=1
for file in filelist[:2]:
    print(f'loading ({i}/{len(filelist)}): {file}...')
    i+=1
    #load .wav
    temp, sr = librosa.load(file, sr=None)
    sf.write('tmp.wav', temp, sr)
    dB_max = 96
    s, fs = maad.sound.load('tmp.wav')
    maxAmp = np.abs(s).max()  # used to normalize
    Sxx_power, tn, fn, ext = maad.sound.spectrogram(
        s/maxAmp, fs, flims=(0, 20000), display=False)
    
    #get ROI from annotation and conver to tn fn
    df_roi_annot = df_annot[df_annot['sound_id']==Path(file).parts[-1][:-4]].reset_index(drop=True)
    df_roi_annot = maad.util.format_features(df_roi_annot,tn,fn)
    #if sound is anotated
    if len(df_roi_annot)>0:
        #compute all shape features
        df_rois_shape = maad.features.all_shape_features(
        s, fs, df_roi_annot, resolution='med', display=False)
        df_rois_shape = df_rois_shape.reset_index(drop=True)
        
        
        df_temporal_features = pd.DataFrame(columns=TEMPORAL_FEATURES)
        df_spectral_features = pd.DataFrame(columns=SPECTRAL_FEATURES)
        for index, row in df_roi_annot.iloc[0:].iterrows():
            df_rois_all_features_temp = pd.DataFrame()
            s_trim = maad.sound.trim(
                s, fs, row.min_t, row.max_t)
            s_trim = s_trim - np.mean(s_trim)
            s_trim = s_trim / np.max(np.abs(s_trim))
            
            try:
                Sxx_trim_power, tn, fn, ext = maad.sound.spectrogram(s_trim, fs,
                                                                        verbose=False, display=False,
                                                                        savefig=None)

                spectral_features_temp, _ = maad.features.all_spectral_alpha_indices(
                        Sxx_trim_power, tn, fn, display=False)
                        
                temporal_features_temp = maad.features.all_temporal_alpha_indices(s_trim, fs)

            except:
                print(str(file)+': error at index (too small?): '+str(index)+'\n s_trim looks like:'+str(s_trim.shape)+'\n temporal size:'+str(row.min_t - row.max_t))

            df_spectral_features = df_spectral_features.append(
                spectral_features_temp, ignore_index=True)
            df_temporal_features = df_temporal_features.append(
                temporal_features_temp, ignore_index=True)
            df_rois_all_features_temp['ID']  = Path(file).parts[-1][:-4]
            df_rois_all_features_temp = pd.concat([df_spectral_features,df_temporal_features],axis=1)
            df_rois_all_features_temp = pd.concat([df_rois_shape,df_rois_all_features_temp],axis=1)
            
        df_rois_all_features_temp = df_rois_all_features_temp.reset_index(drop=True)
        df_annot_final = pd.concat([df_annot_final,df_rois_all_features_temp],axis=0)
    else:
        print('no annotation available')
        df_rois_all_features_temp = pd.DataFrame()
        df_annot_final = pd.concat([df_annot_final,df_rois_all_features_temp],axis=0)
        pass


loading (1/127): /Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/xenocanto/Calendulauda_africanoides/449420.wav...
number of rois : 33
loading (2/127): /Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/xenocanto/Cisticola_galactotes/452901.wav...
no annotation available
loading (3/127): /Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/xenocanto/Cisticola_galactotes/201332.wav...
no annotation available
loading (4/127): /Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/xenocanto/Cisticola_galactotes/710853.wav...
no annotation available
loading (5/127): /Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/xenocanto/Cyanomitra_olivacea/721728.wav...
no annotation available
loading (6/127): /Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/xenocanto/Burhinus_vermiculatus/701253.wav...
number of rois : 2
loading (7/127): /Users/Paul/Paul/Desk

In [187]:
df_annot_final.record.unique()

array([449420, 701253, 448614, 432472, 432471, 432469, 104021, 201496,
       201450,  59742, 452899, 453374, 104058, 721755, 400928, 201315,
       705357, 457592, 444840,  59739])

In [193]:
df_rois_shape

,Unnamed: 0,record,n_roi,min_t,max_t,min_f,max_f,dt,df,APD,...,centroid_y,centroid_x,duration_x,bandwidth_y,area_xy,centroid_f,centroid_t,duration_t,bandwidth_f,area_tf
0,373,59739,1,4.6686,4.8642,471.178,6553.656,0.1956,6082.478,-44.86,...,87.0,410.0,140.991565,0.0,9588,3746.777344,4.771701,0.19737,6072.363281,1198.5


In [200]:
df_annot_final[SPECTRAL_FEATURES]['VARf'][:50]

0     0.000012
1     0.000013
2     0.000152
3     0.000063
4     0.000089
5     0.000089
6     0.000050
7     0.000022
8     0.000088
9     0.000432
10    0.000031
11    0.000058
12    0.000286
13    0.000100
14    0.000117
15    0.000105
16    0.000127
17    0.000023
18    0.000110
19    0.000079
20    0.000244
21    0.000110
22    0.000072
23    0.000109
24    0.000087
25    0.000177
26    0.000062
27    0.000164
28    0.000025
29    0.000081
30    0.000179
31    0.000068
32    0.000099
0     0.000097
1     0.000085
0     0.000142
1     0.000280
2     0.000159
3     0.000126
4     0.000134
5     0.000139
6     0.000219
7     0.000157
8     0.000167
9     0.000212
10    0.000250
11    0.000186
12    0.000211
13    0.000151
14    0.000193
Name: VARf, dtype: float64

In [198]:
df_annot_final[SHAPE_FEATURES]['shp_002']

0     0.154976
1     0.184393
2     0.215874
3     0.186440
4     0.247316
        ...   
12         NaN
13         NaN
0     0.099971
0          NaN
0          NaN
Name: shp_002, Length: 954, dtype: float64

In [119]:
df_annot_final = df_annot_final.reset_index(drop=True)

,Unnamed: 0,record,n_roi,min_t,max_t,min_f,max_f,dt,df,APD,...,BGNt,SNRt,MED,Ht,ACTtFraction,ACTtCount,ACTtMean,EVNtFraction,EVNtMean,EVNtCount
0,814,449420,1,2.9653,3.0856,3787.412,6407.471,0.1203,2620.059,-70.83,...,-5.042324,1.267571,0.559622,0.874052,0.200000,2,4.900824,0.200000,0.02322,9.570312
1,815,449420,2,3.1156,3.3260,4306.235,6485.294,0.2104,2179.059,-68.33,...,-2.772774,-0.169866,0.726804,0.935135,0.000000,0,NaN,0.000000,0.0,0.000000
2,816,449420,3,3.4562,3.5163,3528.000,5058.529,0.0601,1530.529,-62.74,...,-6.717718,2.428221,0.461439,0.749545,0.400000,2,5.409224,0.400000,0.02322,21.533203
3,817,449420,4,3.5564,3.6766,5110.412,7367.294,0.1202,2256.882,-65.42,...,-3.147957,-0.386197,0.696016,0.896978,0.100000,1,3.147957,0.000000,0.0,0.000000
4,818,449420,5,3.6866,3.7668,6407.471,7419.176,0.0801,1011.706,-65.55,...,-0.774764,-0.341514,0.914751,0.980488,0.000000,0,NaN,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,859,705357,13,29.9250,30.5293,1090.385,1999.038,0.6043,908.654,-51.70,...,-12.885848,3.204457,0.226837,0.808304,0.153846,8,9.111395,0.115385,0.06966,1.688879
950,860,705357,14,30.6688,31.3311,1241.827,1968.750,0.6624,726.923,-52.43,...,-2.889905,0.354780,0.716976,0.978043,0.000000,0,NaN,0.000000,0.0,0.000000
951,861,457592,1,0.0939,0.1490,757.212,969.231,0.0551,212.019,-64.99,...,-0.945072,-0.229696,0.899043,0.966997,0.000000,0,NaN,0.000000,0,0.000000
952,799,444840,1,0.6561,0.8516,1968.750,6360.577,0.1955,4391.827,-48.83,...,-5.028194,0.919076,0.560591,0.879022,0.312500,5,4.172872,0.312500,0.05805,5.742187


In [145]:
GENERIC_FEATURES =  ['min_t', 'max_t', 'min_f', 'max_f',
       'dt', 'df', 'min_y', 'min_x', 'max_y', 'max_x','centroid_x', 'duration_x', 'bandwidth_y', 'area_xy', 'centroid_f',
       'centroid_t', 'duration_t', 'bandwidth_f', 'area_tf']

SHAPE_FEATURES = ['shp_002', 'shp_003', 'shp_004', 'shp_005', 'shp_006', 'shp_007',
       'shp_008', 'shp_009', 'shp_010', 'shp_011', 'shp_012', 'shp_013',
       'shp_014', 'shp_015', 'shp_016', 'shp_017', 'shp_018', 'shp_019',
       'shp_020', 'shp_021', 'shp_022', 'shp_023', 'shp_024', 'shp_025',
       'shp_026', 'shp_027', 'shp_028', 'shp_029', 'shp_030', 'shp_031',
       'shp_032', 'shp_033', 'shp_034', 'shp_035', 'shp_036', 'shp_037',
       'shp_038', 'shp_039', 'shp_040', 'shp_041', 'shp_042', 'shp_043',
       'shp_044', 'shp_045', 'shp_046', 'shp_047', 'shp_048']


SPECTRAL_FEATURES = ['MEANf', 'VARf', 'SKEWf', 'KURTf', 'NBPEAKS', 'LEQf',
                        'ENRf', 'BGNf', 'SNRf', 'Hf', 'EAS', 'ECU', 'ECV', 'EPS', 'EPS_KURT', 'EPS_SKEW', 'ACI',
                        'NDSI', 'ROU']

TEMPORAL_FEATURES = ['ZCR', 'MEANt', 'VARt', 'SKEWt', 'KURTt', 'BGNt', 'SNRt', 'MED', 'Ht']

# SPECTRAL_FEATURES = ['MEANf', 'VARf', 'SKEWf', 'KURTf', 'NBPEAKS', 'LEQf',
#                         'ENRf', 'BGNf', 'SNRf', 'Hf', 'EAS', 'ECU', 'ECV', 'EPS', 'EPS_KURT', 'EPS_SKEW', 'ACI',
#                         'NDSI', 'rBA', 'AnthroEnergy', 'BioEnergy', 'BI', 'ROU', 'ADI', 'AEI', 'LFC', 'MFC', 'HFC',
#                         'ACTspFract', 'ACTspCount', 'ACTspMean', 'EVNspFract', 'EVNspMean', 'EVNspCount',
#                         'TFSD', 'H_Havrda', 'H_Renyi', 'H_pairedShannon', 'H_gamma', 'H_GiniSimpson', 'RAOQ',
#                         'AGI']

# TEMPORAL_FEATURES = ['ZCR', 'MEANt', 'VARt', 'SKEWt', 'KURTt',
#                         'LEQt', 'BGNt', 'SNRt', 'MED', 'Ht', 'ACTtFraction', 'ACTtCount',
#                         'ACTtMean']

In [146]:
df_annot_final.to_csv('/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/tables/df_annot_final.csv')

In [149]:
df_annot_final[SHAPE_FEATURES].isna().sum()

shp_002    86
shp_003    86
shp_004    86
shp_005    86
shp_006    86
shp_007    86
shp_008    86
shp_009    86
shp_010    86
shp_011    86
shp_012    86
shp_013    86
shp_014    86
shp_015    86
shp_016    86
shp_017    86
shp_018    86
shp_019    86
shp_020    86
shp_021    86
shp_022    86
shp_023    86
shp_024    86
shp_025    86
shp_026    86
shp_027    86
shp_028    86
shp_029    86
shp_030    86
shp_031    86
shp_032    86
shp_033    86
shp_034    86
shp_035    86
shp_036    86
shp_037    86
shp_038    86
shp_039    86
shp_040    86
shp_041    86
shp_042    86
shp_043    86
shp_044    86
shp_045    86
shp_046    86
shp_047    86
shp_048    86
dtype: int64

In [139]:
pd.read_csv('/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/tables/df_ROI_final.csv')

,x,ROI_id,min_y,min_x,max_y,max_x,min_f,min_t,max_f,max_t,...,Ht,ACTtFraction,ACTtCount,ACTtMean,EVNtFraction,EVNtMean,EVNtCount,gen,species,sound_id
0,0,1,0,343,137,434,0.000000,3.993832,5900.097656,5.050340,...,0.902865,0.246032,31.0,5.892072,0.206349,0.050310,4.134375,Calendulauda,africanoides,449420
1,1,2,4,463,406,589,172.265625,5.387029,17484.960938,6.849887,...,0.862541,0.187500,3.0,4.398765,0.187500,0.034830,5.742187,Calendulauda,africanoides,449420
2,2,3,12,328,31,344,516.796875,3.819683,1335.058594,4.005442,...,0.842645,0.388889,14.0,7.238912,0.388889,0.054180,7.382812,Calendulauda,africanoides,449420
3,3,4,81,87,110,123,3488.378906,1.021678,4737.304688,1.439637,...,0.922238,0.195122,8.0,4.768629,0.170732,0.027090,6.459961,Calendulauda,africanoides,449420
4,4,5,82,12,115,53,3531.445312,0.150930,4952.636719,0.626939,...,0.906963,0.233333,7.0,4.913164,0.233333,0.040635,5.940194,Calendulauda,africanoides,449420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4215,4215,14,135,530,207,565,5813.964844,6.164898,8914.746094,6.571247,...,0.957063,0.136364,3.0,3.546711,0.090909,0.023220,4.101562,Treron,calvus,104019
4216,4216,15,137,304,289,326,5900.097656,3.541043,12446.191406,3.796463,...,0.952035,0.100000,2.0,3.745972,0.000000,0.000000,0.000000,Treron,calvus,104019
4217,4217,16,139,87,276,107,5986.230469,1.021678,11886.328125,1.253878,...,0.938043,0.040000,1.0,3.114189,0.000000,0.000000,0.000000,Treron,calvus,104019
4218,4218,17,357,371,498,396,15374.707031,4.318912,21447.070312,4.609161,...,0.945133,0.043478,1.0,4.814023,0.000000,0.000000,0.000000,Treron,calvus,104019


In [ ]:
df_all_features_roi_temp = compute_ROI_all_features(s, fs, df_rois=df_roi_annot , resolution='med', display=False)

In [ ]:
import numpy as np

In [ ]:
file

In [ ]:
data_filtered_by_annot = df_data[df_data.sound_id.isin(df_annot.sound_id)].reset_index()
fmin= 0 
fmax = 20000

for file in data_filtered_by_annot['fullfilename'][:1]:
    temp, sr = librosa.load(file, sr=None)
    sf.write('tmp.wav', temp, sr)
    dB_max = 96
    s, fs = maad.sound.load('tmp.wav')
    maxAmp = np.abs(s).max()  # used to normalize
    Sxx_power, tn, fn, ext = maad.sound.spectrogram(
        s/maxAmp, fs, flims=(fmin, fmax), display=False)
    Sxx_db = maad.util.power2dB(Sxx_power) + dB_max


    features = ['min_t','max_t','min_f','max_f','dt','df']

    df_roi_annot = df_annot[df_annot['sound_id']==Path(file).parts[-1][:-4]][features]
    df_roi_annot = maad.util.format_features(df_roi_annot,tn,fn)


    print(file)


    df_all_features_roi_temp = compute_ROI_all_features(s, fs, df_rois=df_roi_annot , resolution='med', display=False)

    dff_data = data_filtered_by_annot.copy()
    dff_data = dff_data[dff_data['fullfilename']==file]
    dff_data = dff_data.reset_index()
    df_all_features_roi_temp['gen'] = dff_data['gen'][0]
    df_all_features_roi_temp['species'] = dff_data['species'][0]
    df_all_features_roi_temp['sound_id'] = dff_data['sound_id'][0]
                
    df_all_features_all_roi = df_all_features_all_roi.append(df_all_features_roi_temp)

    
    df_all_features_all_roi = df_all_features_all_roi.reset_index()
    df_all_features_all_roi = df_all_features_all_roi.drop(axis=1,labels=['label','index'])
    df_all_features_all_roi.rename(columns={'labelID': 'ROI_id'}, inplace=True)

    


        
        


In [ ]:
df_roi_annot

In [ ]:
df_data[df_data.sound_id.isin(df_annot.sound_id)]

In [ ]:
def compute_ROI_all_features_all_dataset(datapath=datapath, annot_path=annot_path):
    '''Creates biggest dataframe containing all birds, all roi, all features

    entry dataframe columns: ['fullfilename','gen','species','sound_id']    (all unique ids)
    '''
    filelist = glob(datapath+'/**/*.wav', recursive = True)
    df_data=pd.DataFrame()
    for file in filelist:
        species = Path(file).parts[-2].rsplit(sep='_')[1]
        gen = Path(file).parts[-2].rsplit(sep='_')[0]
        df_data = df_data.append({'fullfilename': file,
                                'sound_id': Path(file).parts[-1][:-4],
                                'species': species,
                                'gen' : gen},
                                ignore_index=True)

    df_all_features_all_annots = pd.DataFrame()

    for filepath in df_data['fullfilename']:
        dff_data = df_data.copy()
        dff_data = dff_data[dff_data['fullfilename']==filepath]
        dff_data = dff_data.reset_index()

        print(filepath)
        try:
            df_roi_temp,s,fs = find_ROIs_soundfile(path=filepath , display=False, mode_bin='relative', param1=10, param2=0.9)
            df_all_features_roi_temp = compute_ROI_all_features(s, fs, df_rois=df_roi_temp , resolution='med', display=False)

            df_all_features_roi_temp['gen'] = dff_data['gen'][0]
            df_all_features_roi_temp['species'] = dff_data['species'][0]
            df_all_features_roi_temp['sound_id'] = dff_data['sound_id'][0]
                
            df_all_features_all_roi = df_all_features_all_roi.append(df_all_features_roi_temp)
        except Exception as e:
            print(e)
    
    df_all_features_all_roi = df_all_features_all_roi.reset_index()
    df_all_features_all_roi = df_all_features_all_roi.drop(axis=1,labels=['label','index'])
    df_all_features_all_roi.rename(columns={'labelID': 'ROI_id'}, inplace=True)
    
    return df_all_features_all_roi, df_data